
# 🌼 제너레이티브AI의 이해 - 1차시(24.11.18)

---


## 1. VAE(Variational Auto Encoder)
- 모델이 숫자 이미지를 압축하고, 복원하는 과정(인코더-디코더)
- 인코더 : 입력 이미지를 잠재공간으로 매핑
- 디코더 : 잠재 공간에서 샘플을 받아 이미지 복원
- 가상환경 실행 후 - pip install opencv-python로 설치!

### Tensorflow를 활용한 celeb데이터셋에 VAE 적용하기

In [41]:
import os
import tensorflow as tf
from tensorflow import keras
import numpy as np
import cv2
import matplotlib.pyplot as plt

In [42]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("yunting0123/img-align-celeba")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\RMARKET\.cache\kagglehub\datasets\yunting0123\img-align-celeba\versions\1


In [43]:
# 데이터 경로 설정
image_dir = "C:/Users/RMARKET/.cache/kagglehub/datasets/yunting0123/img-align-celeba/versions/1/t/celebA"

In [44]:
img_filenames = os.listdir(image_dir)[-1]
img_filenames

'030000.png'

In [45]:
# 1. 이미지 불러오고 전처리 하는 함수 만들기
def load_preprocess_images(image_dir, img_size = (128, 128), num_images = 10000):
    # 경로, 이미지 크기, 최대 로드할 이미지 개수
    images = []
    # 전처리 된 이미지를 추가할 리스트
    img_filenames = os.listdir(image_dir)[:num_images]
    for filename in img_filenames:
        img_path = os.path.join(image_dir, filename)
        # 이미지 파일의 전체 경로를 image_dir(파일 경로) + filename(파일 이름)으로 만들어 주기
        img = cv2.imread(img_path)
        if img is not None:
            img = cv2.resize(img, img_size)
            img = img / 255.0 # 이미지 정규화
            # 정규화 과정
            # rgb [204, 76, 128] -> [0.8, 0.3, 0.5]
            images.append(img)
    return np.array(images)


In [46]:
images = load_preprocess_images(image_dir)
images

array([[[[0.75294118, 0.90588235, 0.99215686],
         [0.75294118, 0.90588235, 0.99215686],
         [0.75294118, 0.90588235, 0.99215686],
         ...,
         [0.78823529, 0.88627451, 0.94901961],
         [0.84705882, 0.92156863, 0.96862745],
         [0.87843137, 0.9372549 , 0.98039216]],

        [[0.75294118, 0.90588235, 0.99215686],
         [0.75294118, 0.90588235, 0.99215686],
         [0.75294118, 0.90588235, 0.99215686],
         ...,
         [0.79607843, 0.89411765, 0.95294118],
         [0.85098039, 0.92156863, 0.97254902],
         [0.87843137, 0.9372549 , 0.98039216]],

        [[0.75294118, 0.90588235, 0.99215686],
         [0.75294118, 0.90588235, 0.99215686],
         [0.75294118, 0.90588235, 0.99215686],
         ...,
         [0.81960784, 0.90588235, 0.96078431],
         [0.85490196, 0.9254902 , 0.97647059],
         [0.8745098 , 0.93333333, 0.98431373]],

        ...,

        [[0.23529412, 0.40392157, 0.6745098 ],
         [0.22745098, 0.39215686, 0.6627451 ]

In [47]:
# 2. 샘플링 함수 정의
# 잠재공간에서 새로운 샘플을 생성하기 위해 데이터포이트들을 샘플링하는 과정
# 각 데이터포인터들의 평균값과 로그 분산을 출력하는 함수
def sampling(z_mean, z_log_var):
    # 모델이 새로운 데이터포인트(z)를 생성할 수 있음
    batch = tf.shape(z_mean)[0]
    # 딥러닝프레임워크는 데이터 입력시 자동으로 배치 크기를 감지
    # 0번째 자리의 값이 batch 사이즈이므로 batch라는 이름의 변수로 저장한 것

    dim = tf.shape(z_mean)[1]
    # 잠재공간의 차원값

    epsilon = tf.random.normal(shape = (batch, dim))
    # 배치사이즈와 차원의 shape을 갖는 표준정규분포의 무작위값 생성

    return z_mean + tf.exp(0.5 * z_log_var) * epsilon
# tf.exp(0.5 z_log_var) -> 로그 분산을 표준편차로 변환하는 과정, 로그분산에 0.5를 곱하고 지수함수를 적용 -> 표준편차를 구함

In [48]:
# 3. 인코더 정의
latent_dim = 200
# 잠재공간의 차원
# 간단한 데이터나 테스트목적 : 2 ~ 10 정도
# 복잡한 이미지나 고차원 : 100 ~ 200 사용

encoder_input = keras.layers.Input(shape = (128, 128, 3), name='encoder_input') # shape = (가로, 세로, 차원(컬러(rgb)))
# 128x128 사이즈의 컬러 이미지를 인풋으로 넣을 것임을 명시적으로 지정한 것
x = keras.layers.Conv2D(32, (3,3), strides = 2, activation = 'relu', padding = 'same')(encoder_input) # padding = 'same' : 가장자리에 패딩이 들어가기 위해(입력과 출력값이 같게 하기 위해)
# Conv2D : 공간적인 패턴 학습 가능
# 32개 필터, (3,3)커널, 슬라이딩 간격 2
x = keras.layers.Conv2D(64, (3, 3), strides = 2, activation = 'relu', padding='same')(x)
x = keras.layers.Conv2D(128,  (3,3), strides = 2, activation= 'relu', padding='same')(x)
# 텐서 : 딥러닝 적인 연산이 가능하다
# x.sahpe = (batch_size, height, width, channels)
shape_before_flattening = x.shape[1:]
# x를 1차원 벡터로 변환하기 전 정보 저장

# 1차원 벡터로 변환 : Conv2D는 이미지의 공간적인 구조를 학습할 수 있으나 결정을 내릴 순 없음
# 결정을 내릴 수 있는 Dense층을 쌓아주기 위해 1차원 벡터로 변환(flatten, 평탄화)
x = keras.layers.Flatten()(x)

# 인코더의 마지막 단계
z_mean = keras.layers.Dense(latent_dim, name = 'z_mean')(x)
z_log_var = keras.layers.Dense(latent_dim, name = 'z_log_var')(x)
# 두 레이어는 동일한 입력 x를 받지만, 손실함수와 역전파 과정에 의해 다른 출력을 학습

z = sampling(z_mean, z_log_var)
# 무작위 벡터값 생성

encoder = keras.models.Model(encoder_input, [z_mean, z_log_var, z], name = 'encoder')
# 모델은 3개의 출력을 반환한다
# z_mean : 인코더가 학습한 잠재 공간의 평균 벡터
# z_log_var : 인코더가 학습한 로그 분산 벡터
# z : 평균과 분산을 사용해 샘플링된 잠재 공간의 벡터


In [49]:
# 4. 디코더 정의
decoder_input = keras.layers.Input(shape = (latent_dim,), name='decoder_input')
x = keras.layers.Dense(np.prod(shape_before_flattening))(decoder_input)
# Dense 레이어로 잠재공간의 벡터를 디코더 출력에 맞게 변환
# prod : 이 형태의 모든 차원을 곱해 1차원 값으로 변환
# (16,16,128) -> 16x16x128
# 먼저 위에서 저장했던 flatten전 마지막 3차원을 1차원으로 변환 후 Dense레이어에 적용

x = keras.layers.Reshape(shape_before_flattening)(x)
# 원래 인코더의 마지막 출력 형태인 3차원 텐서로 다시 reshape
x = keras.layers.Conv2DTranspose(128, (3,3), strides = 2, activation = 'relu', padding='same')(x)
x = keras.layers.Conv2DTranspose(64, (3,3), strides = 2, activation = 'relu', padding='same')(x)
x = keras.layers.Conv2DTranspose(32, (3,3), strides = 2, activation = 'relu', padding='same')(x)

decoder_output = keras.layers.Conv2D(3, (3,3), strides = 1, activation = 'sigmoid', padding='same', name='decoder_output')(x)
decoder = keras.models.Model(decoder_input, decoder_output, name = 'decoder')

In [50]:
optimizer = keras.optimizers.Adam()

In [51]:
# 5. 학습 루프 만들기
def train_step(data):
    with tf. GradientTape() as tape:
        # with tf. GradientTape() : tensorflow에서 자동 미분을 위해 설저하는 with문
        z_mean, z_log_var, z = encoder(data)
        recon = decoder(z)
        # recon : 재구성된 이미지 / 입력데이터와 비교될 이미지
        recon_loss = tf.reduce_mean(500 * tf.losses.binary_crossentropy(data,recon))
        # 원본 데이터와 재구성된 이미지 데이터 간의 차이를
        # 픽셀의 이진분포롤 계산
        # 가중치를 500 곱해서 조정
        # reduce_mean : 배치 내 모든 데이터포인트의 평균손실을 계산한다는 의미

        # 2. KL-발산 손실
        kl_loss = tf.reduce_mean(tf.reduce_sum(-0.5 * (1 + z_log_var - tf.square(z_mean)-tf.exp(z_log_var)), axis = 1)) 
        # KL 발산의 공식 : (1 + z_log_var - tf.square(z_mean)-tf.exp(z_log_var))
        # axis = 1옵션을 넣었기 때문에 각 차원에 대한 합산이 이루어 짐
        total_loss = recon_loss + kl_loss
    grad = tape.gradient(total_loss, encoder.trainable_weights + decoder.trainable_weights) # trainable_weights : 가중치
    optimizer.apply_gradient(zip(grad, encoder.trainable_weights + decoder.trainable_weights)) # 각각의 가중치를 업데이트
    # 계산된 기울기를 사용하여 Adam최적화 알고리즘을 통해 인코더와 디코더의 가중치를 업데이트
    return total_loss, recon_loss, kl_loss
    # 총 손실, 재구성 손실, KL-발산 손실을 반환
        

In [52]:
batch_size = 32
epochs = 10
dataset = tf.data.Dataset.from_tensor_slices(images).shuffle(1000).batch(batch_size)
# tf.data.Dataset.from_tensor_slices : 주어진 텐서를 받아 이를 개별 슬라이스로 나눠 데이터셋을 생성

In [53]:
for epoch in range(epochs):
    print(f'Epoch {epoch + 1} / {epochs}')
    for step, batch_data in enumerate(dataset):
        # 현재 스텝과 배치데이터를 반환
        total_loss, recon_loss, kl_loss = train_step(batch_data)
        if step % 100 == 0:
            # 스텝이 100의 배수일 때 마다
            print(f'step{step} : total loss = {total_loss.numpy():.4f},'
                  f'recon loss : {recon_loss.numpy():.4f}'
                  f'KL loss : {kl_loss.numpy():.4f}')

Epoch 1 / 10


AttributeError: 'Adam' object has no attribute 'apply_gradient'

---
### Torch를 활용한 MNIST데이터셋에 VAE 적용하기

In [ ]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt


## 🌼마무리 문제

다양한 데이터셋을 적용해보세요

Q1. Fashion MNIST
- 28X28 크기의 패션 아이템 이미지로 구성된 흑백 이미지 데이터셋


Q2. CIFAR-10
- 32X32 크기의 색상이 있는 자동차, 동물 등 다양한 객체가 포함된 데이터셋

In [ ]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

# Fashion MNIST 데이터셋 로드 (28x28 크기의 패션 아이템 이미지)
trans = transforms.ToTensor()
fashion_mnist_data = datasets.FashionMNIST(root='./data', train=True, transform=trans, download=True)
data_loader = DataLoader(fashion_mnist_data, batch_size=64, shuffle=True)



In [ ]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

# CIFAR-10 데이터셋 로드 (32x32 크기의 컬러 이미지)
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # 데이터 정규화
])
cifar10_data = datasets.CIFAR10(root='./data', train=True, transform=transform, download=True)
data_loader = DataLoader(cifar10_data, batch_size=64, shuffle=True)
